In [ ]:
import numpy as np
import trimesh

def cartesian_to_spherical(v):
    x, y, z = v
    phi = np.arctan2(y, x)
    mu = z  # cos(theta)
    return [phi, mu]

# 生成五角化十二面体（Cardinal polyhedron）
pentakis = trimesh.creation.icosphere(subdivisions=0)
# pentakis = pentakis.subdivide_to_size(max_edge=0.99)  # 控制面数（细化得到大约60面）

# 投影到单位球面
vertices = pentakis.vertices
vertices /= np.linalg.norm(vertices, axis=1)[:, None]

# 提取每个三角形的球面顶点坐标 (phi, mu)
result = []
for face in pentakis.faces:
    verts = [vertices[i] for i in face]
    area = np.linalg.norm(np.cross(verts[1] - verts[0], verts[2] - verts[0]))/2
    # print(area, np.sqrt(3)/4 * np.linalg.norm(verts[1] - verts[0])**2)
    spherical_coords = [cartesian_to_spherical(v) for v in verts]
    result.append((spherical_coords,area))


print(len(result))
# 输出为 C++ static constexpr 形式
print("static constexpr std::array<S2Cell, 20> s2_cells = {")
for tri,area in result:
    print("    S2Cell{{ vector2f{{ {:.16f}, {:.16f} }}, vector2f{{ {:.16f}, {:.16f} }}, vector2f{{ {:.16f}, {:.16f} }}, {:.16f} }},"
          .format(tri[0][0], tri[0][1], tri[1][0], tri[1][1], tri[2][0], tri[2][1], area))
print("};")
